In [1]:
import re
import os
from Bio import SeqIO
import pandas as pd

In [2]:
# read file

mmseqs_res_path = "/home/guangyichen/Desktop/WIBI/phd/Tasks/Phylogeny/Phylogeny-redo/SI0.7_threshold50/mmseqs_res/clusterRes_0.7_cluster.tsv"
tsv_read = pd.read_csv(mmseqs_res_path, header = None, delim_whitespace=True)
representative_proteins = tsv_read[0]
all_proteins = tsv_read[1]

In [3]:
# Select Protein Clusters
# [Curated Data] 
# In total, threshold = 50, SI = 0.5, 350997 selected 
#           threshold = 50, SI = 0.6, 323989 selected 
#           threshold = 50, SI = 0.7, 268488 selected 

# Select Protein Clusters
# [AF Data] 


threshold = 20 # Curated: 50, AF: 15?

protein_list = [] 
#protein_cluster_list = []

for i in range(0,all_proteins.size-1):
    if (representative_proteins[i] != "-"):
        each_cluster = []
        each_cluster.append(representative_proteins[i])
        for j in range(i+1,representative_proteins.size-1):
            if (representative_proteins[j] == representative_proteins[i]):
                each_cluster.append(all_proteins[j])
                representative_proteins[j] = "-"
            else:
                break

        clades_list = []
        for k in range(0, len(each_cluster)-1):
            each_protein = each_cluster[k]
            #print(each_protein)
            
            #'''
            # For Curated Data
            if (each_protein[0:4] == "HIPS"):
                each_clade = re.sub("_BOGUS_[0-9]{2,}", "", each_protein)
                each_clade = re.sub("_[a-z]{2,}.{2,}$", "", each_clade)
            elif (each_protein[0:3] == "SBCy"):
                each_clade = re.sub("_JPGINDPC_[0-9]{2,}", "", each_protein)
            else:
                each_clade = re.sub(".gb.{2,}", "", each_protein)
                each_clade = re.sub("_[a-z | A-Z].*.", "", each_clade)
            
            #'''
    
            # For AF Data: 
            #each_clade = re.sub("-.{1,}$","",each_protein)
            
            clades_list.append(each_clade)

        clades_set = set(clades_list) 
        
        if (len(clades_set) >= threshold):
            #protein_cluster_list.append(each_cluster)
            
            for each_protein in each_cluster:
                protein_list.append(each_protein)
                
            #print("----")

print(len(protein_list))
#print(protein_list)

298750


In [4]:
# Get Unique Clade 
# For testing purpose 

all_clade_names = []
    
for each_protein in protein_list: 
    #'''
    if (each_protein[0:4] == "HIPS"):
        record_clade_name = re.sub("_BOGUS_[0-9]{2,}", "", each_protein)
        record_clade_name = re.sub("_[a-z]{2,}.{2,}$", "", record_clade_name)
    elif (each_protein[0:3] == "SBCy"):
        each_clade = re.sub("_JPGINDPC_[0-9]{2,}", "", each_protein)
    else:
        record_clade_name = re.sub(".gb.{2,}", "", each_protein)
        record_clade_name = re.sub("_[a-z | A-Z].*.", "", record_clade_name)
    #'''
        
     # For AF Data: 
    #record_clade_name = re.sub("-.{1,}$","",each_protein)        
        
        
    all_clade_names.append(record_clade_name)
    
    
unique_clade_name = set(all_clade_names)
#list(unique_clade_name)
print(len(unique_clade_name))
#print(unique_clade_name)



57


In [17]:
# Output Selected Proteins as Fasta file 

output_dir = "/home/guangyichen/Desktop"
input_protein_folder = "/home/guangyichen/Desktop/WIBI/phd/Tasks/Phylogeny/Phylogeny-redo/All-Proteins" 

counter = 1
selected_prot = ""

for each_protein in protein_list:
    
    #'''
    if (each_protein[0:4] == "HIPS"):
        each_clade = re.sub("_BOGUS_[0-9]{2,}", "", each_protein)
        each_clade = re.sub("_[a-z]{2,}.{2,}$", "", each_clade)
    elif (each_protein[0:3] == "SBCy"):
        each_clade = re.sub("_JPGINDPC_[0-9]{2,}", "", each_protein)
    else:
        each_clade = re.sub(".gb.{2,}", "", each_protein)
        each_clade = re.sub("_[a-z | A-Z].*.", "", each_clade)
    
    each_clade = each_clade + ".fasta"
    #'''
    
    # For AF Data: 
    #each_clade = re.sub("-.{1,}$",".faa",each_protein) 
    
    input_protein_fasta = os.path.join(input_protein_folder, each_clade) 
    for seq_record in SeqIO.parse(input_protein_fasta, "fasta"):
        if (seq_record.id == each_protein):
            record_protein = ">" + seq_record.id + "\n" + seq_record.seq + "\n"
            selected_prot += record_protein
            #print(counter/len(protein_list)*100, end = "\r")
            print("{0:.0%}".format(counter/len(protein_list)), end = "\r")
            counter += 1
            break 
        else:
            pass 
    

out_f_name = "Selected_Prot.fasta"
with open(os.path.join(output_dir, out_f_name), "w") as file:
    file.write(str(selected_prot))